# AutoGenU for Jupyter

An Automatic C++ Code Generator for Nonlinear Model Predictive Control (NMPC)  

---  

## Formulation of the optimal control problem: Mobile robot

### Model description
<img src="https://raw.github.com/wiki/mayataka/CGMRES/images/fig_mobilerobot.png" width=40%>

The above figure shows the model of a mobile robot. The kinematics of the robot is given by
$$ \dot{X} = V \cos{\theta}, $$
$$ \dot{Y} = V \sin{\theta}. $$
Physical limitations of the robot are given by
$$ V_{\rm min} \leq V \leq V_{\rm max}, $$
$$ w_{\rm min} \leq \dot{\theta} \leq w_{\rm max}. $$

### Control Objective
We consider tracjectory tracking problem. We set the reference trajectory as 
$$ x_{\rm ref} (t) = \begin{bmatrix} t v_{\rm ref} \\ 0 \end{bmatrix} .$$
We also assume that there are two circular obstacles to be avoided whose center and radius are $(X_1, Y_1)$, $R_1$ and $(X_2, Y_2)$, $R_2$, respectively.

### Formulation of the optimal control problem
We define the state vector as
$$ x = \begin{bmatrix} X \\ Y \\ \theta \end{bmatrix} .$$
We assume that $V=u_1$ and $\theta = u_2$. The state equation is then given as 
$$ \dot{x} = f(t, x, u) = \begin{bmatrix} u_1 \cos{\theta} \\ u_1 \sin{\theta} \\ u_2 \end{bmatrix} $$

Next, we set the cost function 
$$ J = \varphi(t, x) + \int_{t}^{t+T} L(t, x, u) .$$
To achieve the aforementioned goal, we set the terminal cost as
$$\varphi(t,x) = \frac{1}{2} q_1 (x_1 - t v_{\rm ref})^2 + \frac{1}{2} q_2 x_2 ^2 + \frac{1}{2} q_3 x_3 ^2 $$  
and the stage cost as 
$$ L(t, x, u) = \frac{1}{2} q_1 (x_1 - t v_{\rm ref})^2 + \frac{1}{2} q_2 x_2 ^2 + \frac{1}{2} q_3 x_3 ^2 + \frac{1}{2} r_1 (u_1 \cos{x_3} - v_{\rm ref})^2 + \frac{1}{2} r_2 u_2 ^2 $$  
where $q_1, q_2, q_3$, $r_1, r_2$ are positive real constant.
From the physical limitations and the obstacles, there are 6 inequality constraints 
$$
h(x, u) = \begin{bmatrix} R_1 ^2 - (x_1 - X_1)^2 - (x_2 - Y_1)^2 \\
                          R_2 ^2 - (x_1 - X_2)^2 - (x_2 - Y_2)^2 \\
                          v_{\rm min} - u_1 \\
                          u_1 - v_{\rm max} \\ 
                          w_{\rm min} - u_2 \\ 
                          u_2 - w_{\rm max} 
                          \end{bmatrix} \leq 0
$$
In this example, these inequality constraints are considered by the semi-smooth Fischer-Burmeister method.
In the optimization problem, the complementary conditions are imposed for inequality constraints, e.g., for $g(x, u) \in \mathbb{R}$,
$$
    g(x, u) \leq 0 ,
$$
$$
    \nu \geq 0 ,
$$
$$
    \nu g(x, u) = 0 .
$$
The semi-smooth Fischer-Burmeister method transforms these conditions into
$$
    \Phi(\nu, - g(x, u)) = 0, \;\; \Phi(a, b) = \sqrt{a^2 + b^2 + \epsilon ^2} - (a + b) .
$$
where $\epsilon > 0$ is a regularization term. 
After all, in the optimal control problem, we find the solution satisfying the following conditions
$$
    \left( \frac{\partial H}{\partial u} \right)^{\rm T} = \left( \frac{\partial L}{\partial u} \right)^{\rm T} + \left( \frac{\partial f}{\partial u} \right)^{\rm T} \lambda + \left( \frac{\partial h}{\partial u} \right)^{\rm T} \nu,
$$
$$
    \Phi(\nu_i, - h_i(x, u)) = 0 ,
$$
where $\lambda$ and $\nu$ are the Lagrange multipliers and the latter condition is imposed on elementwise.

### References 
For more details about the semi-smooth Fischer-Burmeister method in the optimal control problem, especially with the C/GMRES method, see the following references.
- [M. Huang, et. al, Nonlinear Model Predictive Control of a Diesel Engine Air Path: A Comparison of Constraint Handling and Computational Strategies, IFAC-PapersOnLine, Vol. 48, No. 23, pp. 372-379 (2015)](https://doi.org/10.1016/j.ifacol.2015.11.308)
- [M. Huang, Low Complexity Model Predictive Control of a Diesel Engine Airpath, Ph. D. diss., University of Michigan (2016)](https://deepblue.lib.umich.edu/handle/2027.42/120832)
- [S. Alamdari, S. Amin, Stochastic Model Predictive Control for Eco-Driving Assistance Systems in Electric Vehicles, Ph. D. diss., University of Luxembourg (2018)](http://orbilu.uni.lu/handle/10993/36164)
- [D. Liao-McPherson, M. Huang and I. Kolmanovsky, A Regularized and Smoothed Fischer–Burmeister Method for Quadratic Programming With Applications to Model Predictive Control, IEEE Transactions on Automatic Control, Vol. 64, No. 7, pp. 2937-2944 (2019)](https://doi.org/10.1109/TAC.2018.2872201)

### Numerical solver of NMPC
In the following example, we compute the solution of NMPC using the multiple-shooting-based C/GMRES method.

---

# Generate C++ codes of NMPC model  

## Import modules

In [ ]:
from sympy import *
from autogenu import symbolic_functions as symfunc
from autogenu import cpp_generator as gencpp

## Set dimensions
`dimx`: Dimension of the state vector $x$   
`dimu`: Dimension of the control input vector $u$  
`dimc`: Dimension of the equality constraints $C(x, u) = 0$  
`dimh`: Dimension of the inequality constraints $h(x, u) \leq 0$ considered using the semi-smooth Fischer-Burmeister method

In [ ]:
dimx = 3
dimu = 2
dimc = 0
dimh = 6

## Generate t, x, u, and lmd, necessary variables for the formulation of the optimal control problem
`t`: Time parameter $t$  
`x`: The state vector $x$  
`u`: The control input vector $u$  
`lmd`: The Lagrange multiplier for the state equation $\lambda$

In [ ]:
t = Symbol('t')  
x = symbols('x[0:%d]' %(dimx))  
u = symbols('u[0:%d]' %(dimu+dimc+dimh))
lmd = symbols('lmd[0:%d]' %(dimx))

## Define variables used in the state equation, constraints, and cost function
- Define a scalar variable whose name is "var" as  
`var = Symbol('var') `

- Define scalar variables whose names are "var\_1", "var\_2", ..., "var\_n" as  
`var_1, var_2, ..., var_n = symbols('var_1, var_2, ..., var_n')`

- Define a vector whose name is "vec" and dimension is dim_vec as  
`vec = symbols(f'vec[0:{dim_vec}]')`

In [ ]:
# Define user variables used in the state function here
vx_ref = Symbol('vx_ref')
v_min, v_max = symbols('v_min, v_max')
w_min, w_max = symbols('w_min, w_max')
X_1, Y_1, R_1 = symbols('X_1, Y_1, R_1')
X_2, Y_2, R_2 = symbols('X_2, Y_2, R_2')
xx_ref = vx_ref * t

## Define weight parameters used in the stage cost and the terminal cost
`q`: The weight array for the state in the stage cost  
`r`: The weight array for the control input in the stage cost  
`q_terminal`: The weight array for the state in the terminal cost  
`x_ref`: The reference value of the state  
`fb_eps`: The regularization term for the semi-smooth Fischer-Burmeister method

In [ ]:
q = symbols('q[0:%d]' %(dimx))
r = symbols('r[0:%d]' %(dimu+dimc))
q_terminal = symbols('q_terminal[0:%d]' %(dimx))
x_ref = symbols('x_ref[0:%d]' %(dimx))
fb_eps = symbols('fb_eps[0:%d]' %(dimh))

## Define the state equation, constraints, the stage cost, and the terminal cost
`fxu`: The state equation $ f(t, x, u)$  
`Cxu`: The equality constraisnts $C(t, x, u) = 0 $  
`hxu`: The inequality constraisnts $h(t, x, u) \leq 0$  
`L`: The stage cost $L(t, x, u)$  
`phi`: The terminal cost $\phi (t, x)$  
Note: array indices start with 0

In [ ]:
# Define the state equation
fxu = [u[0] * cos(x[2]),
       u[0] * sin(x[2]),
       u[1]]
    
# Define the equality constraints C(x, u) = 0 (if dimc > 0)
Cxu = []

# Define the inequality constraints h(x, u) <= 0 (if dimh > 0) considered using the semi-smooth Fischer-Burmeister method
hxu = [R_1**2 - (x[0]-X_1)**2 - (x[1]-Y_1)**2,
       R_2**2 - (x[0]-X_2)**2 - (x[1]-Y_2)**2,
       v_min - u[0],
       u[0] - v_max,
       w_min - u[1],
       u[1] - w_max]

# Define the stage cost
L = (q[0]*(x[0]-xx_ref)**2 + q[1]*x[1]**2 + q[2]*x[2]**2) / 2 + (r[0]*(u[0]*cos(x[2])-vx_ref)**2 + r[1]*u[1]**2) / 2

# Define the terminal cost
phi = (q[0]*(x[0]-xx_ref)**2 + q[1]*x[1]**2 + q[2]*x[2]**2)/2 

## Generate the optimality conditions
`Hamiltonian`: $H(t, x, u, \lambda, \mu, \nu) := L(t, x, u) + \lambda^{\rm T} f(t, x, u) + \mu^{\rm T} C(x, u) + \nu^{\rm T} h(x, u) $  
`hx`: The partial derivartive of the Hamiltonian $H (t, x, u, \lambda, \mu, \nu)$ with respect to $x$, i.e., $\left(\frac{\partial H}{\partial x} \right)^{\rm T} (t, x, u, \lambda, \mu, \nu)$    
`hu`: The partial derivartive of the Hamiltonian $H (t, x, u, \lambda, \mu, \nu)$ with respect to $(u, \mu, \nu)$, i.e., $\left(\frac{\partial H}{\partial (u, \mu, \nu)} \right)^{\rm T}, (t, x, u, \lambda, \mu, \nu)$. Note that the last `dimh` elements are composed by $\Phi_{\epsilon} (\nu_i, -h_i(x,u))$ where $\Phi_{\epsilon} (a, b) = \sqrt{a^2 + b^2 + \epsilon} - (a+b)$  
`phix`: The partial derivative of the terminal cost $\phi(x)$ with respect to $x$, i.e., $\left(\frac{\partial \phi}{\partial x} \right)^{\rm T} (t, x)$

In [ ]:
Hamiltonian = L + symfunc.dot_product(lmd, fxu) + sum(u[dimu+i] * Cxu[i] for i in range(dimc)) + sum(u[dimu+dimc+i] * hxu[i] for i in range(dimh))
phix = symfunc.diff_scalar_func(phi, x)
hx = symfunc.diff_scalar_func(Hamiltonian, x)
hu = symfunc.diff_scalar_func(Hamiltonian, u)
for i in range(dimh):
    hu[dimu+dimc+i] = sqrt(u[dimu+dimc+i]**2 + hxu[i]**2 + fb_eps[i]) - (u[dimu+dimc+i] - hxu[i])

## Symplify phix, hx, and hu
Note: if `hx` and `hu` is too complicated, it takes too much time to simplify them

In [ ]:
phix = simplify(phix)
hx = simplify(hx)
hu = simplify(hu)

## Set Parameters
Set all parameters used in the state equation, constraints, and the cost function

In [ ]:
scalar_parameters = [[vx_ref, 0.4], 
                     [X_1, 1], [Y_1, 0.25], [R_1, 0.5],
                     [X_2, 2], [Y_2, -0.25], [R_2, 0.5],
                     [v_min, -0.5], [v_max, 0.5],
                     [w_min, -0.75], [w_max, 0.75]]

array_parameters = [['q', dimx, '{10, 1, 0.01}'], 
                    ['r', dimu, '{0.1, 0.1}'], 
                    ['q_terminal', dimx, '{10, 1, 0.1}'], 
                    ['x_ref', dimx, '{0, 0, 0}'],
                    ['fb_eps', dimh, '{0.01, 0.01, 0.0, 0.0, 0.0, 0.0}']]

## Generate C++ codes of NMPC model
Generate `nmpc_model.hpp` and `nmpc_model.cpp`, C++ source files of NMPC problem settings.  

`model_name`: Name of the directory where `nmpc_model.hpp` and `nmpc_model.cpp` are generated.  
`cse_flag`: The flag for common subexpression elimination. If `True`, common subexpressions in fxu, phix, hx, and hu are eliminated when `nmpc_model.cpp` is generated. Default is `Flase`. If the symbolic functions are too complicated, it may take time.

In [ ]:
model_name = "mobilerobot"
cse_flag = True

gencpp.make_model_dir(model_name)
gencpp.generate_cpp(fxu, phix, hx, hu, model_name, cse_flag)
gencpp.generate_hpp(dimx, dimu, dimc+dimh, scalar_parameters, array_parameters, model_name)

---  
# Generate C++ codes for numerical simulation  

## Import modules

In [ ]:
from autogenu import solver_parameters as slvprm
from autogenu import initialization_parameters as iniprm
from autogenu import simulation_parameters as simprm
from autogenu import cpp_executor as cppexe
from autogenu import simulation_plottor as simplt
from autogenu import animation_generator as animgen
init_printing()

## Set solvers  
set which solvers you use  in `solver_index`
  
1.  `ContinuationGMRES` : The continuation/GMRES method (the original C/GMRES method, single shooting).
2.  `MultipleShootingCGMRES` : The multiple shooting based C/GMRES method with condensing of the state and the Lagragne multipliers with respect to the state equation.
3.  `MSCGMRESWithInputSaturation` : The multiple shooting based C/GMRES method with condensing of the state, the Lagragne multipliers with respect to the state equation, and variables with respect to the constraints on the saturation function on the control input.

In [ ]:
solver_index = 2

## Set saturaions on the control input if you choose `solver_index = 3`
- Saturation on the control input: $u_{i, {\rm min}} \leq u_i \leq u_{i, {\rm max}}$  
$u_i \in \mathbb{R}$ : a constrained component of the control input $u$  
- Transformed equality constraint: $(u_i - \frac{u_{i, {\rm max}} + u_{i, {\rm min}}}{2})^2 - ( \frac{u_{i, {\rm max}} - u_{i, {\rm min}}}{2})^2 + {u_d}_i ^2 = 0$  
${u_d}_i \in \mathbb{R}$ : a dummy input for the transformation of the saturation into the equality constraint  
- Additional term in the stage cost $L(x, u)$ with respect to the saturation of $u_i$: $- {r_d}_i {u_d}_i + \frac{1}{2} {r_q}_i {u_d}_i ^2$  
   ${r_d}_i > 0$: a weight parameter to avoid failure of numerical computation, ${r_q}_i \geq 0$: a weight parameter to increase mergin of the saturation  

`index`: $i$  
`u_min`: $u_{i, {\rm min}}$  
`u_max`: $u_{i, {\rm max}}$   
`dummy_weight`: ${r_d}_i > 0$  
`quadratic_weight` :  ${r_q}_i \geq 0$  
`saturation_param` = [`index`, `u_min`, `u_max`, `dummy_weight`, `quadratic_weight`]

In [ ]:
saturation_list = []
# Define saturation_list as follows.
# saturation_list = [[index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight], 
#                    .., 
#                    [index , u_min, u_max, dummy_weight, quadratic_weight]

## Set parameters for the solver

`T_f`, `alpha`: Parameters for the length of the horizon $T(t)$: $T(t) = T_f (1 - e^{-\alpha t})$.  
`N`: The division number of the horzion for the numerical computation.  
`finite_difference_increment`: Step length of a finite difference approximations of hessian-vector products in C/GMRES.   
`zeta`: A stabilization parameter of the C/GMRES method. It may work well to set `zeta` by the reciprocal of the sampling time.  
`kmax`: The maximam number of the iteration of the GMRES.

In [ ]:
T_f = 1.5
alpha = 1.0
N = 50
finite_difference_increment = 1.0e-08
zeta = 1000  # zeta must be the reciprocal of the sampling period
kmax = 15

solver_parameters = slvprm.SolverParameters(T_f, alpha, N, finite_difference_increment, zeta, kmax)

## Set parameters for the initialization of the solution  
`initial_guess_solution`: The initial guess of the solution of the optimal control problem (OCP) for initialization of the solution of NMPC.   
`newton_residual_torelance`: The residual torelance of the solution of the OCP for the initialization of the solution of NMPC. The Newton iteration terminates when the optimality error is less than this velue.  
`max_newton_iteration`: The maxmum number of Newton iteration for the initialization of the solution of NMPC.  
`initial_Lagrange_multiplier`: An optional parameter for `MSCGMRESWithInputSaturation`. This is a part of the initial guess of the solution, the initial guess of the Lagrange multiplier with respect the constraints on the saturation function of the control input. NOTE that this parameter has to be defined as list. If you do not specify this parameter, the all of initial Lagrange multiplier are set by 1e-03.

In [ ]:
initial_guess_solution = [0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
newton_residual_torelance = 1.0e-06
max_newton_iteration = 50

if solver_index == 1 or solver_index == 2:
    initialization_parameters = iniprm.InitializationParameters(initial_guess_solution, 
                                                                newton_residual_torelance, 
                                                                max_newton_iteration)
else:
    initialization_parameters = iniprm.InitializationParameters(initial_guess_solution, 
                                                                newton_residual_torelance, 
                                                                max_newton_iteration, 
                                                                Lagrange_multiplier)

## Set parameters for numerical simulation
`initial_time`: Initial time of the numerical simulation.  
`initial_state`: Initial state vector of the system.  
`simulation_time`: Simulation time of the numerical simulation.  
`sampling_time`: The sampling time of the numerical simulation.

In [ ]:
initial_time = 0  
initial_state = [0, 0, 0]  
simulation_time = 10 
sampling_time = 0.001  

simulation_parameters = simprm.SimulationParameters(initial_time, 
                                                    initial_state, 
                                                    simulation_time, 
                                                    sampling_time)

## Generate main.cpp and CMakeLists.txt

In [ ]:
if solver_index == 1 or solver_index == 2:
    gencpp.generate_main(solver_index, 
                         solver_parameters, 
                         initialization_parameters, 
                         simulation_parameters, 
                         model_name)
else:
    gencpp.generate_main(solver_index, 
                         solver_parameters, 
                         initialization_parameters, 
                         simulation_parameters, 
                         model_name, 
                         saturation_list)    

gencpp.generate_cmake(solver_index, model_name)
gencpp.generate_cmake_for_model(model_name)

## Build and run simulation
NOTE: If you use Windows OS and an error occurs in `cppexec.setCMake(model_name)`, you may solve that error by running codes which are commented out instead of the original codes.

In [ ]:
cppexe.set_cmake(model_name)
cppexe.make_and_run(model_name)
# cppexe.remove_build_dir(model_name)
# cppexe.set_cmake(model_name, MSYS=True)
# cppexe.make_and_run(model_name)

## Plot the simulation results

In [ ]:
plot = simplt.SimulationPlottor(model_name)
plot.set_scales(2,5,2)
# plot.show_plots()
plot.save_plots()

## Draw an animation of the simulation results

In [ ]:
vx_ref = 0.4
X1, Y1, R1 = 1, 0.25, 0.5
X2, Y2, R2 = 2, -0.25, 0.5

anim = animgen.MobileRobot(model_name, vx_ref, X1, Y1, R1, X2, Y2, R2)
anim.set_skip_frames(10)
anim.generate_animation()